## Introduction
    In this weeks experiment we will be using a grid world with a goal of finding a policy that given any starting position will lead to a positive reward exit.

## Experiment
Words

## Conclusion
Words

## Appendix


In [77]:
from reinforce.utils.rlgridworld.standard_grid import create_standard_grid, create_negative_grid, create_large_grid
from reinforce.utils.rlgridworld.algorithms import iterative_policy_evaluation, compute_policy_from_values

ImportError: cannot import name 'create_large_grid' from 'reinforce.utils.rlgridworld.standard_grid' (C:\Users\Tyler\reinforcement-learning-gym\reinforce\utils\rlgridworld\standard_grid.py)

### Value Iteration

In [ ]:
# from page 83 of Sutton and Barto, RL 2nd. Ed.
def value_iteration(gw, gamma=0.9, epsilon=0.001):
    count = 0
    while True:
        count += 1
        biggest_change_in_value = 0
        for node in gw:
            state = node.state
            if not gw.is_terminal(state) and not gw.is_barrier(state):
                old_value = gw.get_value(state)
                new_value = float('-inf')
                # valid decisions and rewards at current state
                dr = gw.valid_decisions_and_rewards(state)
                for action, reward in dr.items():
                    reward = gw.get_reward_for_action(state, action)
                    value_at_dest = gw.get_value_at_destination(state, action)
                    value = reward + gamma*value_at_dest
                    if value > new_value:
                        new_value = value
                    gw.set_value(state, new_value)
                biggest_change_in_value = max(biggest_change_in_value,
                                                  abs(new_value - old_value))
        if biggest_change_in_value < epsilon:
            break

In [ ]:
gw = create_standard_grid()

print("")
print("Initial Values")
gw.print_values()

# compute values
value_iteration(gw)

print("")
print("Values after Value Iteration")
gw.print_values()

# compute policy from values
policy = compute_policy_from_values(gw)

print("")
print("New Policy")
gw.print_policy(policy)

In [ ]:
gw = create_negative_grid()

print("")
print("Initial Values")
gw.print_values()

# compute values
value_iteration(gw)

print("")
print("Values after Value Iteration")
gw.print_values()

# compute policy from values
policy = compute_policy_from_values(gw)

print("")
print("New Policy")
gw.print_policy(policy)

### Policy Iteration

In [ ]:
gw = create_standard_grid()

In [ ]:
policy = {
    (0,0):'right', (0,1):'right',(0,2):'right',(0,3):'up',
    (1,0):'up', (1,1):'', (1,2):'right', (1,3):'',
    (2,0):'right', (2,1):'right', (2,2):'right', (2,3):''
    }

In [ ]:
# from page 80 of Sutton and Barto, RL, 2nd. Ed.
def policy_iteration(gw, policy, gamma=0.9, epsilon=0.001):
    while True:
        # perform iterative policy evaluation to update values
        iterative_policy_evaluation(gw, policy, gamma, epsilon)
        # update policy from new values
        new_policy = compute_policy_from_values(gw, gamma)
        # see if policy has changed
        for action in policy:
            if policy[action] == new_policy[action]:
                policy_stable = True
            else:
                policy_stable = False
                break
        # update policy
        policy = new_policy
        # repeat until policy does not change
        if policy_stable == True:
            break

In [ ]:
print("")
print("Initial Policy")
gw.print_policy(policy)
print("")

# note: this execution of iterative policy evaluation is not part
# of the policy iteration algorithm.  It is for the purpose of
# displaying the values associated with the input policy

iterative_policy_evaluation(gw, policy)
print("Initial Policy Values")
gw.print_values()

# run policy iteration algorithm
policy_iteration(gw, policy)
# compute policy from optimal values
new_policy = compute_policy_from_values(gw)

# print new policy and values
print("")
print("New Policy")
gw.print_policy(new_policy)
print("")
print("New Policy Values")
gw.print_values()
print("")

In [ ]:
gw = create_negative_grid()

policy = {
    (0,0):'right', (0,1):'right',(0,2):'right',(0,3):'up',
    (1,0):'up', (1,1):'', (1,2):'right', (1,3):'',
    (2,0):'right', (2,1):'right', (2,2):'right', (2,3):''
    }

print("")
print("Initial Policy")
gw.print_policy(policy)
print("")

# note: this execution of iterative policy evaluation is not part
# of the policy iteration algorithm.  It is for the purpose of
# displaying the values associated with the input policy

iterative_policy_evaluation(gw, policy)
print("Initial Policy Values")
gw.print_values()

# run policy iteration algorithm
policy_iteration(gw, policy)
# compute policy from optimal values
new_policy = compute_policy_from_values(gw)

# print new policy and values
print("")
print("New Policy")
gw.print_policy(new_policy)
print("")
print("New Policy Values")
gw.print_values()
print("")

### Iterative Policy Evaluation

In [ ]:
gw = create_standard_grid()

In [ ]:
policy = {
    (0,0):'up', (0,1):'right',(0,2):'right',(0,3):'up',
    (1,0):'up', (1,1):'', (1,2):'right', (1,3):'',
    (2,0):'right', (2,1):'right', (2,2):'right', (2,3):''
    }

In [ ]:
print("Policy")
gw.print_policy(policy)
print("Initial Values")
gw.print_values()

In [ ]:
def iterative_policy_evaluation(gw, policy, gamma=0.9, theta=0.001):

    while True:
        biggest_change = 0
        for node in gw:
            state = node.state
            if not gw.is_terminal(state) and not gw.is_barrier(state):
                # get current (old) value
                old_value = gw.get_value(state)
                # get action from policy
                action = policy[state]
                # get immediate reward for action
                reward = gw.get_reward_for_action(state, action)
                # get value at destination state
                value_at_dest = gw.get_value_at_destination(state, action)
                # compute new value
                new_value = reward + gamma*value_at_dest
                # set new value for state
                gw.set_value(state, new_value)
                # see if |new_value-old_value| is larger than biggest_change
                biggest_change = max(
                    biggest_change, abs(new_value-old_value))
        # iterated over all states, so see if biggest_change is small enough
        if biggest_change < theta:
            break

In [ ]:
print("Policy")
gw.print_policy(policy)
iterative_policy_evaluation(gw, policy, gamma = 0.9)
print("Values for the policy")
gw.print_values()

In [ ]:
gw = create_negative_grid()

policy = {
    (0,0):'up', (0,1):'right',(0,2):'right',(0,3):'up',
    (1,0):'up', (1,1):'', (1,2):'right', (1,3):'',
    (2,0):'right', (2,1):'right', (2,2):'right', (2,3):''
    }

print("Policy")
gw.print_policy(policy)
iterative_policy_evaluation(gw, policy, gamma = 0.9)
print("Values for the policy")
gw.print_values()

### Policy from Values

In [ ]:
gw = create_standard_grid()

In [ ]:
policy = {
    (0,0):'up', (0,1):'right',(0,2):'right',(0,3):'up',
    (1,0):'up', (1,1):'', (1,2):'right', (1,3):'',
    (2,0):'right', (2,1):'right', (2,2):'right', (2,3):''
    }
print("Input Policy")
gw.print_policy(policy)

In [ ]:
iterative_policy_evaluation(gw, policy, gamma = 0.9)

In [ ]:
print("Values for the input policy")
gw.print_values()

In [ ]:
def compute_policy_from_values(gw, gamma = 0.9):
    # create null policy dictionary
    policy = {}
    # loop over all states
    for i in range(gw.M):
        for j in range(gw.N):
            state = (i,j)
            # assign 'no' policy to barrier states, there are no actions at barrier states
            if gw.is_barrier(state):
                policy[state] = ''
            # assign 'no' policy to terminal sttes, there are no actions at terminal states
            if gw.is_terminal(state):
                policy[state] = ''
            # for all non terminal and non barrier states
            if not gw.is_terminal(state) and not gw.is_barrier(state):
                # set candidate best action and best value
                best_action = None
                best_value = float('-inf')
                # get dictionary of all valid decisions and rewards at current state (i,j)
                dr = gw.valid_decisions_and_rewards(state)
                # iterate over all action, reward in
                for action, reward in dr.items():
                    # get reward for current action
                    reward = gw.get_reward_for_action(state,action)
                    # get the value of the destination state for the current action
                    value_at_dest = gw.get_value_at_destination(state,action)
                    # compute candidate vale
                    value = reward + gamma*value_at_dest
                    # if value is better, then update best action and best value
                    if value > best_value:
                        best_value = value
                        best_action = action
                # add best action to the policy dictionary
                policy[state] = best_action
    return policy

In [ ]:
new_policy = compute_policy_from_values(gw)

In [ ]:
print("Original Policy")
gw.print_policy(policy)
print("")
print("New Policy")
gw.print_policy(new_policy)

In [ ]:
gw = create_negative_grid()

policy = {
    (0,0):'up', (0,1):'right',(0,2):'right',(0,3):'up',
    (1,0):'up', (1,1):'', (1,2):'right', (1,3):'',
    (2,0):'right', (2,1):'right', (2,2):'right', (2,3):''
    }
print("Input Policy")
gw.print_policy(policy)

iterative_policy_evaluation(gw, policy, gamma = 0.9)

new_policy = compute_policy_from_values(gw)

print("Original Policy")
gw.print_policy(policy)
print("")
print("New Policy")
gw.print_policy(new_policy)

### Large Grid Experiment

In [ ]:
gw = create_large_grid()

policy = {
    (0,0):'up', (0,1):'right',(0,2):'right',(0,3):'up', (0, 4): 'left',
    (1,0):'up', (1,1):'', (1,2):'right', (1,3):'down', (1, 4): 'down',
    (2,0):'right', (2,1):'right', (2,2):'right', (2,3):'', (2,4): 'down',
    (3,0): 'down', (3,1): 'right', (3,2): 'right', (3,3): 'right', (3,4): 'down',
    (4,0): 'down', (4,1): 'right', (4,2): 'right', (4,3): '', (4,4): 'left',
}
print("Input Policy")
gw.print_policy(policy)

iterative_policy_evaluation(gw, policy, gamma = 0.9)

new_policy = compute_policy_from_values(gw)

print("Original Policy")
gw.print_policy(policy)
print("")
print("New Policy")
gw.print_policy(new_policy)
